In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# CONFIG
VECTORIZER_PICKLE = 'vectorizer.pkl'
CLUSTERING_PICKLE = 'clustering.pkl'
INDUSTRIES_LIST = '../industries.txt'

unlabeled_data = '../unlabeled.csv'

data_types = {
    'id': np.int64,
    'text': str,
}

In [2]:
%%time
print('Reading data')
data = pd.read_csv(unlabeled_data, usecols=['id', 'text'], dtype=data_types, engine='c', index_col='id')
data.text.replace(np.nan, "", inplace=True)
print('Number of samples texts: ', len(data))

Reading data
Number of samples texts:  200000
CPU times: user 8.99 s, sys: 1.46 s, total: 10.4 s
Wall time: 13.9 s


In [3]:
%%time
vectorizer = pickle.load(open(VECTORIZER_PICKLE, 'rb'))

CPU times: user 4.25 s, sys: 1.36 s, total: 5.61 s
Wall time: 23.4 s


In [4]:
%%time
clustering = pickle.load(open(CLUSTERING_PICKLE, 'rb'))

CPU times: user 3.84 ms, sys: 8.28 ms, total: 12.1 ms
Wall time: 30.6 ms


In [5]:
data['word_vec'] = data.text.apply(lambda x: vectorizer.transform([x]))
data.head(10)

,text,word_vec
id,,
1500001,Home-Luxury Move Management 0 Skip to Content ...,"(0, 66887)\t0.03480269167294948\n (0, 66049..."
1500002,Specialty Insurance | Surety Bonds | Intact In...,"(0, 66906)\t0.014428428830911926\n (0, 6689..."
1500003,Home | The Association of International Proper...,"(0, 66981)\t0.04531469282729924\n (0, 66416..."
1500004,The domain name getroxo.com is for sale Excell...,"(0, 66887)\t0.018062972137871596\n (0, 6676..."
1500005,DH Marrion | Providing electrical contracts to...,"(0, 66851)\t0.06539549890991274\n (0, 66416..."
1500006,"Mobile Makeup, Hair & Beauty Services Online |...","(0, 66434)\t0.013250909680298491\n (0, 6638..."
1500007,"Norton Consulting & Investigation - Security, ...","(0, 66766)\t0.02494398361227716\n (0, 66403..."
1500008,"Laurel Wealth Planning financial advisors, Edi...","(0, 66911)\t0.03925055248383764\n (0, 66887..."
1500009,Audit and Recovery Services HomeServicesTroubl...,"(0, 66753)\t0.013940394806553368\n (0, 6658..."


In [11]:
data['clusterID'] = data.word_vec.apply(lambda x: clustering.predict(x)[0] + 1)

KeyError: ('id', 'clusterID')

In [25]:
data['temp'] = data.clusterID.apply(lambda x:x[0])
data.rename(columns={'clusterID': 'temp1', 'temp': 'clusterID'}, inplace=True)
data

TypeError: 'int' object is not subscriptable

In [30]:
data.clusterID.to_csv('company2cluster_206312506.csv', header=True)